# YouTube Face Detection

This script is designed to download a YouTube video, detect faces in the video frames, and create an output video with the detected faces highlighted. It uses the `pytube`, `moviepy`, and `OpenCV` libraries for video processing and face detection. The code is organized into functions to perform specific tasks, such as downloading the video, parallelizing face detection, and cleaning up temporary files. It is powered by the Ray framework for parallel and distributed computing.

Note that the sound is not preserved in the output video.

## Imports and Setup

In [ ]:
import os
import ray_ease as rez
import cv2
import moviepy.editor
import pytube
import warnings

warnings.filterwarnings("ignore")

# Set the MODEL_PATH environment variable to the Haar Cascade model's location
tmp = os.getenv("MODEL_PATH")
os.environ["MODEL_PATH"] = tmp if tmp else "./haarcascade_frontalface_default.xml"

OUTPUT_DIR = "./tmp"

## Functions Definition

In [ ]:
@rez.parallelize
def detect_faces(fn, start, stop):
    # Extract the subclip from the video
    clip = moviepy.editor.VideoFileClip(fn).subclip(start, stop)

    # Load face detector
    face_cascade = cv2.CascadeClassifier(os.getenv("MODEL_PATH"))

    # Run face detector on frames
    imgs = []
    for img in clip.iter_frames():
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        for x, y, w, h in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
        imgs.append(img)

    # Create mp4 of result
    out_clip = moviepy.editor.ImageSequenceClip(imgs, fps=clip.fps)
    out_fn = f"./clips/{start:04d}.mp4"
    out_clip.write_videofile(out_fn, logger=None)
    return out_fn

def process_video(url, limit = None):
    print(f"Downloading video from '{url}'")
    yt = pytube.YouTube(url)
    stream = yt.streams.filter(file_extension="mp4").first()
    fn = stream.download(output_path="./clips/", max_retries=5)

    # Get duration
    duration = moviepy.editor.VideoFileClip(fn).duration

    # Create (start, stop) intervals
    intervals = [(fn, offset, offset + 1) for offset in range(int(duration))]
    if limit:
        intervals = intervals[:limit]

    print("Processing each range of 1s intervals in parallel using Ray")
    out_fns = rez.retrieve([detect_faces(*interval) for interval in intervals], parallel_progress=True, parallel_progress_kwargs={"desc": "1s Intervals"})

    print("Converting detections to video clips")
    out_clips = [moviepy.editor.VideoFileClip(out_fn) for out_fn in out_fns]

    print("Concatenating results")
    final_clip = moviepy.editor.concatenate_videoclips(out_clips)
    final_fn = "./clips/out.mp4"
    final_clip.write_videofile(final_fn, logger=None)

    # Return the full image data
    with open(final_fn, "rb") as f:
        return os.path.basename(fn), f.read()
    
def main(youtube_url: str = "https://www.youtube.com/watch?v=dQw4w9WgXcQ", output_dir = OUTPUT_DIR, limit = None):
    fn, movie_data = process_video(youtube_url, limit=limit)
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    abs_fn = os.path.join(output_dir, f"{os.getenv('RAY_EASE')}_"+fn)
    print(f"writing results to {abs_fn}")
    with open(abs_fn, "wb") as f:
        f.write(movie_data)

def clean(folder = "./clips/"):
    print("Cleaning clips directory.")

    for i in os.listdir(folder):
        os.remove(os.path.join(folder, i))
    os.rmdir(folder)
    

## Functions Run

In [ ]:
import time

start = time.time()

rez.init("ray")

# To process a smaller part of the video use the `limit` argument and set it
# to the number of 1s interval you want to process, by default `limit=None`
# to process the complete video
main()

total = time.time() - start
print(f"duration = {int((total//60)//60)}h {int((total//60)%60)}min {int(total%60)}s")

clean()

## Get the clips

In [ ]:
!ls clips

In [ ]:
!cp clips/000*.mp4 content/local

## Further Directions

As you can tell from the resulting video, this face detection model is not state of the art. It has plenty of false positives (non-faces being labeled faces) and false negatives (real faces not being labeled). For better model, consider a modern one based on deep learning.